## **데이터베이스 이해**
### **1. Sqlite 이해하기**
#### **[Dbeaver 설치하고 Sqlite DB 확인](https://dbeaver.io/)**
### **2. Data flows like stream with Buffer**
### **3. WIth statement 이해**

In [1]:
! pip install faker

In [2]:
import sqlite3
import pandas as pd
from faker import Faker
import random

# Initialize Faker instance
fake = Faker()

# Connect to SQLite (creates the database file)
conn = sqlite3.connect('flight_booking.db')
cursor = conn.cursor()

# 인스턴스의 개수
sample_siz= 1000


In [3]:
# Ensure all tables are created before inserting data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Users (
        Email TEXT PRIMARY KEY
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Passengers (
        ID INTEGER PRIMARY KEY,
        Name TEXT,
        Email TEXT,
        Gender TEXT,
        Age INTEGER,
        FOREIGN KEY(Email) REFERENCES Users(Email)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Bookings (
        ID INTEGER PRIMARY KEY,
        User_Email TEXT,
        Booking_Date TEXT,
        FOREIGN KEY(User_Email) REFERENCES Users(Email)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Flights (
        Code TEXT PRIMARY KEY
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Journeys (
        ID INTEGER PRIMARY KEY,
        Flight_Code TEXT,
        Date_Of_Journey TEXT,
        Booking_ID INTEGER,
        FOREIGN KEY(Flight_Code) REFERENCES Flights(Code),
        FOREIGN KEY(Booking_ID) REFERENCES Bookings(ID)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS BP_Relationships (
        Booking_ID INTEGER,
        Passenger_ID INTEGER,
        FOREIGN KEY(Booking_ID) REFERENCES Bookings(ID),
        FOREIGN KEY(Passenger_ID) REFERENCES Passengers(ID)
    )
''')


In [4]:

# Function to generate fake users
def generate_fake_users(n=sample_siz):
    users = []
    for _ in range(n):
        users.append((fake.email(),))
    return users

# Function to generate fake passengers
def generate_fake_passengers(users, n=sample_siz):
    passengers = []
    for i in range(n):
        name = fake.name()
        email = random.choice(users)[0]  # Randomly select from generated users
        gender = random.choice(['Male', 'Female'])
        age = random.randint(18, 80)
        passengers.append((i + 1, name, email, gender, age))
    return passengers

# Function to generate fake bookings
def generate_fake_bookings(users, n=sample_siz):
    bookings = []
    for i in range(n):
        user_email = random.choice(users)[0]
        booking_date = fake.date_this_year()
        bookings.append((i + 1, user_email, booking_date))
    return bookings

# Function to generate fake flights
def generate_fake_flights(n=sample_siz):
    flights = []
    for _ in range(n):
        flight_code = fake.bothify('FL###')  # Generate flight code like FL123
        flights.append((flight_code,))
    return flights

# Function to generate fake journeys
def generate_fake_journeys(bookings, flights, n=sample_siz):
    journeys = []
    for i in range(n):
        flight_code = random.choice(flights)[0]
        date_of_journey = fake.date_this_year()
        booking_id = random.choice(bookings)[0]
        journeys.append((i + 1, flight_code, date_of_journey, booking_id))
    return journeys

# Function to generate fake BP relationships
def generate_fake_bp_relationships(bookings, passengers, n=sample_siz):
    bp_relationships = []
    for _ in range(n):
        booking_id = random.choice(bookings)[0]
        passenger_id = random.choice(passengers)[0]
        bp_relationships.append((booking_id, passenger_id))
    return bp_relationships


In [5]:
# Generate fake data
users_data = generate_fake_users(sample_siz)
passengers_data = generate_fake_passengers(users_data, sample_siz)
bookings_data = generate_fake_bookings(users_data, sample_siz)
flights_data = generate_fake_flights(sample_siz)
journeys_data = generate_fake_journeys(bookings_data, flights_data, sample_siz)
bp_relationships_data = generate_fake_bp_relationships(bookings_data, passengers_data, sample_siz)

# Insert data into SQLite tables
cursor.executemany('INSERT OR IGNORE INTO Users (Email) VALUES (?)', users_data)
cursor.executemany('INSERT OR IGNORE INTO Passengers VALUES (?, ?, ?, ?, ?)', passengers_data)
cursor.executemany('INSERT OR IGNORE INTO Bookings VALUES (?, ?, ?)', bookings_data)
cursor.executemany('INSERT OR IGNORE INTO Flights (Code) VALUES (?)', flights_data)
cursor.executemany('INSERT OR IGNORE INTO Journeys VALUES (?, ?, ?, ?)', journeys_data)
cursor.executemany('INSERT OR IGNORE INTO BP_Relationships VALUES (?, ?)', bp_relationships_data)


# Commit changes
conn.commit()


In [6]:

# Load tables into pandas DataFrames
df_users = pd.read_sql_query('SELECT * FROM Users', conn)
df_passengers = pd.read_sql_query('SELECT * FROM Passengers', conn)
df_bookings = pd.read_sql_query('SELECT * FROM Bookings', conn)
df_flights = pd.read_sql_query('SELECT * FROM Flights', conn)
df_journeys = pd.read_sql_query('SELECT * FROM Journeys', conn)
df_bp_relationships = pd.read_sql_query('SELECT * FROM BP_Relationships', conn)

# Store data as dictionary
dataframes_dict = {
    'Users': df_users,
    'Passengers': df_passengers,
    'Bookings': df_bookings,
    'Flights': df_flights,
    'Journeys': df_journeys,
    'BP_Relationships': df_bp_relationships
}

# Close connection
conn.close()


In [7]:
for table_name, df in dataframes_dict.items():
  print(f"--- {table_name} ---")
  display(df.head())

--- Users ---


,Email
0,wbaldwin@example.net
1,leeangela@example.net
2,jonesvicki@example.com
3,ismith@example.net
4,melody88@example.net


--- Passengers ---


,ID,Name,Email,Gender,Age
0,1,Angela Harris,owheeler@example.com,Female,37
1,2,Juan Hanson,annereynolds@example.com,Female,32
2,3,Sharon Martinez,mclaughlinkathy@example.org,Male,71
3,4,Theresa Moore,ijames@example.com,Female,75
4,5,Mrs. Dawn Torres,daniel09@example.com,Female,53


--- Bookings ---


...

,Code
0,FL840
1,FL889
2,FL124
3,FL597
4,FL648


--- Journeys ---


,ID,Flight_Code,Date_Of_Journey,Booking_ID
0,1,FL561,2024-03-15,767
1,2,FL473,2024-10-04,692
2,3,FL779,2024-04-12,885
3,4,FL080,2024-06-30,479
4,5,FL561,2024-04-24,206


--- BP_Relationships ---


,Booking_ID,Passenger_ID
0,736,240
1,168,143
2,180,635
3,665,563
4,300,495


In [8]:
import pandas as pd

# Perform joins to combine all data into one DataFrame
df_combined = df_bp_relationships \
    .merge(df_bookings, left_on='Booking_ID', right_on='ID', suffixes=('_bp', '_booking')) \
    .merge(df_passengers, left_on='Passenger_ID', right_on='ID', suffixes=('_booking', '_passenger')) \
    .merge(df_users, left_on='Email', right_on='Email') \
    .merge(df_journeys, left_on='Booking_ID', right_on='Booking_ID') \
    .merge(df_flights, left_on='Flight_Code', right_on='Code')

# Remove duplicate columns if needed
df_combined.drop(columns=['ID_booking', 'ID_passenger', 'Code'], inplace=True)


In [9]:
df_combined

,Booking_ID,Passenger_ID,User_Email,Booking_Date,Name,Email,Gender,Age,ID,Flight_Code,Date_Of_Journey
0,736,240,jamesnewman@example.org,2024-05-06,Brooke Ferguson,conwaymichael@example.org,Male,69,127,FL184,2024-01-18
1,736,240,jamesnewman@example.org,2024-05-06,Brooke Ferguson,conwaymichael@example.org,Male,69,132,FL713,2024-10-01
2,736,240,jamesnewman@example.org,2024-05-06,Brooke Ferguson,conwaymichael@example.org,Male,69,160,FL932,2024-07-09
3,736,240,jamesnewman@example.org,2024-05-06,Brooke Ferguson,conwaymichael@example.org,Male,69,580,FL696,2024-06-05
4,168,143,kevinsmith@example.org,2024-04-29,James Chang,pattersonandrea@example.net,Female,64,268,FL647,2024-02-08
...,...,...,...,...,...,...,...,...,...,...,...
1975,926,568,marybowen@example.org,2024-07-14,Larry Garcia,kimberlycunningham@example.net,Male,59,856,FL364,2024-04-13
1976,926,568,marybowen@example.org,2024-07-14,Larry Garcia,kimberlycunningham@example.net,Male,59,1000,FL348,2024-09-11
1977,93,901,gonzalezmichael@example.org,2024-03-16,Patricia Robinson,lstewart@example.net,Male,35,560,FL849,2024-05-07
1978,893,354,smithrandall@example.com,2024-03-03,Sarah Thomas,iscott@example.org,Male,51,234,FL215,2024-06-14


In [10]:
pd.read_sql_query('SELECT * FROM Users', conn)

ProgrammingError: Cannot operate on a closed database.

In [11]:
conn = sqlite3.connect('flight_booking.db')
cursor = conn.cursor()

In [12]:
pd.read_sql_query('SELECT * FROM Users', conn)

,Email
0,wbaldwin@example.net
1,leeangela@example.net
2,jonesvicki@example.com
3,ismith@example.net
4,melody88@example.net
...,...
1985,smithdominique@example.com
1986,christinehall@example.org
1987,qhendricks@example.org
1988,millerjose@example.net


In [13]:
conn.close()

## Cntext Manager
### With function ~ 구문

In [14]:
# prompt: with 구문으로 읽기

import sqlite3
import pandas as pd

conn = sqlite3.connect('flight_booking.db')

with conn:
  df_users = pd.read_sql_query('SELECT * FROM Users', conn)
  df_passengers = pd.read_sql_query('SELECT * FROM Passengers', conn)
  df_bookings = pd.read_sql_query('SELECT * FROM Bookings', conn)
  df_flights = pd.read_sql_query('SELECT * FROM Flights', conn)
  df_journeys = pd.read_sql_query('SELECT * FROM Journeys', conn)
  df_bp_relationships = pd.read_sql_query('SELECT * FROM BP_Relationships', conn)

pd.read_sql_query('SELECT * FROM Users', conn)

,Email
0,wbaldwin@example.net
1,leeangela@example.net
2,jonesvicki@example.com
3,ismith@example.net
4,melody88@example.net
...,...
1985,smithdominique@example.com
1986,christinehall@example.org
1987,qhendricks@example.org
1988,millerjose@example.net


In [15]:
pd.read_sql_query('SELECT * FROM Passengers', conn)

,ID,Name,Email,Gender,Age
0,1,Angela Harris,owheeler@example.com,Female,37
1,2,Juan Hanson,annereynolds@example.com,Female,32
2,3,Sharon Martinez,mclaughlinkathy@example.org,Male,71
3,4,Theresa Moore,ijames@example.com,Female,75
4,5,Mrs. Dawn Torres,daniel09@example.com,Female,53
...,...,...,...,...,...
995,996,Jennifer Aguilar,joseph73@example.com,Female,42
996,997,Linda Bender,brent72@example.org,Male,65
997,998,Roy Dawson,millermartha@example.org,Female,22
998,999,Ricky Melendez,lauramartin@example.net,Female,49


In [16]:
pd.read_sql_query('SELECT * FROM Passengers', conn)

,ID,Name,Email,Gender,Age
0,1,Angela Harris,owheeler@example.com,Female,37
1,2,Juan Hanson,annereynolds@example.com,Female,32
2,3,Sharon Martinez,mclaughlinkathy@example.org,Male,71
3,4,Theresa Moore,ijames@example.com,Female,75
4,5,Mrs. Dawn Torres,daniel09@example.com,Female,53
...,...,...,...,...,...
995,996,Jennifer Aguilar,joseph73@example.com,Female,42
996,997,Linda Bender,brent72@example.org,Male,65
997,998,Roy Dawson,millermartha@example.org,Female,22
998,999,Ricky Melendez,lauramartin@example.net,Female,49


In [17]:
conn.close()

In [18]:
with sqlite3.connect('flight_booking.db') as conn:
  df_users = pd.read_sql_query('SELECT * FROM Users', conn)
  display(df_users)

,Email
0,wbaldwin@example.net
1,leeangela@example.net
2,jonesvicki@example.com
3,ismith@example.net
4,melody88@example.net
...,...
1985,smithdominique@example.com
1986,christinehall@example.org
1987,qhendricks@example.org
1988,millerjose@example.net


In [19]:
pd.read_sql_query('SELECT * FROM Passengers')

TypeError: read_sql_query() missing 1 required positional argument: 'con'

In [20]:
pd.read_sql_query('SELECT * FROM Passengers', conn)

,ID,Name,Email,Gender,Age
0,1,Angela Harris,owheeler@example.com,Female,37
1,2,Juan Hanson,annereynolds@example.com,Female,32
2,3,Sharon Martinez,mclaughlinkathy@example.org,Male,71
3,4,Theresa Moore,ijames@example.com,Female,75
4,5,Mrs. Dawn Torres,daniel09@example.com,Female,53
...,...,...,...,...,...
995,996,Jennifer Aguilar,joseph73@example.com,Female,42
996,997,Linda Bender,brent72@example.org,Male,65
997,998,Roy Dawson,millermartha@example.org,Female,22
998,999,Ricky Melendez,lauramartin@example.net,Female,49
